<a href="https://colab.research.google.com/github/pedro00dk/python-lml/blob/master/tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow

This notebook contains my progress studing the TensorFlow library.


### Resources
* https://www.tensorflow.org/guide/
* https://www.tensorflow.org/api_docs/
* https://learningtensorflow.com/getting_started/


In [0]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import tensorflow as tf

## Basics

Tensorflow has two main components, they are:
* `tf.Graph`: Builds the computational graph;
* `tf.Session`,: Runs the computational graph.

### The Computational Graph: `tf.Graph`

A computational graph is a series of TensorFlow operations, organized in a graph, and this graph is made by two elements:
* `tf.Operation`: Operations are the nodes of the graph, these elements produce and consume Tensors.
* `tf.Tensor`: Tensors are the edges of the graph, they represent the values flow over the graph, but they do not store any value.

*Most of TensorFlow operations create an Operation (graph node) and returns its output Tensor (node output edge), the operation node can be accessed through *`tensor.op`* property of the output Tensor*


The most simple element in TensorFlow is a `tf.constant`, it represents a simple operation that does not take any tensor as input but the input passed as argument.


In [0]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly

# c = tf.add(a, b) # this is an alias to tf.math.add
c = a + b # plus operator implements the previous line

print('Operations -> a:', repr(a.op), ', b:', repr(b.op), ', c:', repr(c.op))
print('Tensors -> a:', a, ', b:', b, ', c:', c)
print(a.graph, b.graph, c.graph)
print(tf.get_default_graph())

Operations -> a: <tf.Operation 'Const' type=Const> , b: <tf.Operation 'Const_1' type=Const> , c: <tf.Operation 'add' type=Add>
Tensors -> a: Tensor("Const:0", shape=(), dtype=float32) , b: Tensor("Const_1:0", shape=(), dtype=float32) , c: Tensor("add:0", shape=(), dtype=float32)
<tensorflow.python.framework.ops.Graph object at 0x7f6ffaf4c3c8> <tensorflow.python.framework.ops.Graph object at 0x7f6ffaf4c3c8> <tensorflow.python.framework.ops.Graph object at 0x7f6ffaf4c3c8>


The code above does not output the expected result `7.0`,  it only builds the computation graph where the Tensors represents the output of the operations that will be run.

As we can see, `a` and `b` variables are output Tensors of the `const` operation. When we sum these variables, the result is also a new output Tensor for a new `add` operation, the created `add` operation took the `a` and `b` tensors as input, they should have the same type. The add output tensor has the same type of `a` and `b`.

The Operations and Tensors receive an unique name, this name is not correlated with the python variables names. Operations names starts with the operation type folowed by an unique incremental index e. g. `Const_1`, the output tensor of this operation contains the operation name folowed by it's output index (an operation can output more than one Tensor) e. g. `Const_1:0`.

`a`, `b` and `c` are inside the same graph, the tensorflow default graph, it also can be accessed through `tf.get_default_graph()`.

### The Executor: `tf.Session`

The session is responsible for running the computational graph and return to the user the result of the selected tensors. Assesion can be initialized with the `tf.Session` function. When the session receives a request to compute the output of a Tensor through the `tf.Session.run` method, TensorFlow navigates backwards in the graph to find all nodes that provide some input and then starts the computation.

The function `tf.Session.run` can receive a single tensor to get the output, or a combination of tuples, lists and tuples with tensors as values, a structure with the same layout will be returned with the tensors results.

In [0]:
session = tf.Session()
print(session.run(c))
print(session.run({'constants in tuple': (a, b), 'constants in list': [a, b], 'constants in dict': {'a': a, 'b':b}, 'add': c}))

7.0
{'constants in tuple': (3.0, 4.0), 'constants in list': [3.0, 4.0], 'constants in dict': {'a': 3.0, 'b': 4.0}, 'add': 7.0}


Each call to `tf.Session.run` gets an unique value of the input operations, **if a tensor is accessed more than once in the same run call, it  will always produce the same output value. But in different run calls it may produce different values.**
An example is `tr.random_uniform`, it's value is computed every `tf.Session.run` call, but if it's accessed twice in the same run call, its value will not be recomputed.

In [0]:
rand_vec = tf.random_uniform(shape=(3,))

print('different run calls: ', session.run(rand_vec), session.run(rand_vec))
print('same run call: ', session.run((rand_vec, rand_vec)))
print('same run call (derivated tensors): ', session.run((rand_vec + 1, rand_vec + 2)))

different run calls:  [0.4013343  0.69642997 0.67386544] [0.07974064 0.9824977  0.5567194 ]
same run call:  (array([0.7635484 , 0.14407098, 0.14349544], dtype=float32), array([0.7635484 , 0.14407098, 0.14349544], dtype=float32))
same run call (derivated tensors):  (array([1.3784289, 1.6924123, 1.5190382], dtype=float32), array([2.378429 , 2.6924124, 2.5190382], dtype=float32))


Session should be closed to release resources through the method `tf.Session.close`. `tf.Session` also implements the methods `__enter__` and `__exit__`, making the use of python's `with` keyword available.

In [0]:
session.close()

with tf.Session() as session:
    pass

### Feeding

The graphs that we saw are not very useful because they always produce the same result,. A graph can be parametrized, it means the value of some of its input nodes can be provided later (accepting external input), these nodes are known as placeholders and can be created with the `tf.placeholder` function.

In [0]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
c = a + b

print('Operations -> a:', repr(a.op), ', b:', repr(b.op), ', c:', repr(c.op))
print('Tensors -> a:', a, ', b:', b, ', c:', c)

Operations -> a: <tf.Operation 'Placeholder' type=Placeholder> , b: <tf.Operation 'Placeholder_1' type=Placeholder> , c: <tf.Operation 'add_3' type=Add>
Tensors -> a: Tensor("Placeholder:0", dtype=float32) , b: Tensor("Placeholder_1:0", dtype=float32) , c: Tensor("add_3:0", dtype=float32)


The code above produced two `tf.placeholder` operations and tensors.

The placeholders values should be provided in the `tf.Session.run` call, the parameter **`feed_dict`**, receives a dict with the placeholders objects as keys and the placeholder assigned value as value, the value may be a single element but can be any iterable too, in this case, if there are more than one placeholder, those values shall have the same dimension, and in every dimension, the children shall have the same shape.

In [0]:
with tf.Session() as session:
    print(session.run(c, feed_dict={a: 3, b: 4.5}))
    print(session.run(c, feed_dict={a: [1, 3], b: [2, 4]}))
    print(session.run(c, feed_dict={a: [[[1, 2], [3, 4]], [[1, 2],[1, 2]]], b: [[[4, 3], [2, 1]], [[2, 1], [1, 2]]]}))

7.5
[3. 7.]
[[[5. 5.]
  [5. 5.]]

 [[3. 3.]
  [2. 4.]]]


### Datasets and Iterators

TensorFlow placeholders as useful for small experiments and maybe some graph parameters, but according to TensorFlow's website, `tf.data` is the prefered method of streaming data to a model.

The `tf.data.Dataset.from_tensor_slices` creates a dataset from the received data, where the elements are the slices ate the 0th dimension of the received data, the elements shall have the same shape (dense vector).

`tf.data.Dataset.make_one_shot_iterator` and `tf.data.Dataset.make_initializable_iterator` provide an `tf.data.Iterator` of the dataset slices. There are other types of ways to initialize an iterator thar provides iterators with different properties.

In [0]:
data = [
    [0, 1],
    [2, 3],
    [4, 5],
    [6, 7]
]
dataset = tf.data.Dataset.from_tensor_slices(data)

# datasets also can be initialized from a tensor too, including placeholders
# dataset = tf.data.Dataset.from_tensor_slices(tf.constant(data))

iterator = dataset.make_one_shot_iterator()

print(dataset, iterator)

<TensorSliceDataset shapes: (2,), types: tf.int32> <tensorflow.python.data.ops.iterator_ops.Iterator object at 0x7f6ffa6f5390>


The tensorflow iterator doesnot work like other iterators, you need to produce only the first element, each consecutive call to `tf.Session.run` requiring the tensor generated by the first `tf.data.Iterator.get_next` call will provide the next slice element from the dataset. If the slice tensor is required more than once in a single `tf.Session.run` call, the values will not be updated.

In [0]:
slice = iterator.get_next()
print(slice)

with tf.Session() as session:
    try:
        # while True: print(session.run(slice))
        while True: print(session.run((slice, slice))) # using the tensor twice produces the same result
    except tf.errors.OutOfRangeError as e:
        pass

Tensor("IteratorGetNext:0", shape=(2,), dtype=int32)
(array([0, 1], dtype=int32), array([0, 1], dtype=int32))
(array([2, 3], dtype=int32), array([2, 3], dtype=int32))
(array([4, 5], dtype=int32), array([4, 5], dtype=int32))
(array([6, 7], dtype=int32), array([6, 7], dtype=int32))


One shot iterators are not reusable, but it's possible to create a reusable iterator for the same dataset with the `tf.data.Dataset.make_initializable_iterator`, this method returns an unitialized iterator that shall be initialized by the session using the `tf.data.Iterator.initializer` property. The `tf.data.Iterator.initializer` can be used to initialize the iterator as many times it is necessary.

In [0]:
data = [
    [0, 1],
    [2, 3],
    [4, 5],
    [6, 7]
]
dataset = tf.data.Dataset.from_tensor_slices(data)
iterator = dataset.make_initializable_iterator()

print(dataset, iterator)

slice = iterator.get_next()
print(slice)

with tf.Session() as session:
    session.run(iterator.initializer) # initializes the iterator
    try:
        while True: print(session.run(slice), end=' ')
    except tf.errors.OutOfRangeError as e:
        pass

<TensorSliceDataset shapes: (2,), types: tf.int32> <tensorflow.python.data.ops.iterator_ops.Iterator object at 0x7f6ff6de5d30>
Tensor("IteratorGetNext_1:0", shape=(2,), dtype=int32)
[0 1] [2 3] [4 5] [6 7] 

Combine iterators and placeholders is a good strategy to use new data for different `tf.Session.run` calls.

In [0]:
data_0 = [0, 1, 2, 3]
data_1 = [4, 5, 6, 7]

placeholder = tf.placeholder(tf.float32)
dataset = tf.data.Dataset.from_tensor_slices(placeholder)
iterator = dataset.make_initializable_iterator()

print(dataset, iterator)

with tf.Session() as session:

    print('first session')
    session.run(iterator.initializer, feed_dict={placeholder: data_0}) # initializes the iterator using the placeholder data
    slice = iterator.get_next()
    try:
        while True: print(session.run(slice), end=' ')
    except tf.errors.OutOfRangeError as e:
        pass
    print()

    print('second session')
    session.run(iterator.initializer, feed_dict={placeholder: data_1}) # initializes the iterator using the placeholder data, again
    slice = iterator.get_next()
    try:
        while True: print(session.run(slice), end=' ')
    except tf.errors.OutOfRangeError as e:
        pass
    print()

<TensorSliceDataset shapes: <unknown>, types: tf.float32> <tensorflow.python.data.ops.iterator_ops.Iterator object at 0x7f6ffaf3fbe0>
first session
0.0 1.0 2.0 3.0 
second session
4.0 5.0 6.0 7.0 


### Layers

In order to train a model, the trainning process must modify model parameters and get differet results for the same input durring the trainning process.

`tf.layers` are the recomended way to add treinable parameters to a computational graph, this modules packages the necessary `tf.Variable`s and `tf.Operation`s that manipulate them.

The `tf.layers.Dense` if a good example, it performs a weighted sum across all inputs for each output and applies an optional activation function. The connection weights and biases are managed by the layer object.

#### Creating and Initalizing Layers

In order treate a tensorflow layer tensor, we have to create a `tf.layers.Layer` object. The `tf.layers.Dense` is one of them, it takes some layer properties like the dimensionality of the output and returns a `tf.layers.Layer` object.

The `tf.layers.Layer` is a **callable** object, when called with the input tensor, it checks the tensor output dimensionality and returns the layer tensor.

For each `tf.layers.Dense` like objects, tensorflow provides a **functional interface** e. g. `tf.layers.dense` that receives the input tensor as first argument and then the `tf.layers.Dense` arguments. This approach has some advantages and desavantages. Using the functional interface reduces the amount of code necessary, but the debugging is harder and it's impossible to reuse a pre-made customized `tf.layers.Layer` object.

The `tf.layers.Dense` implements the operation **outputs = activation(inputs * kernel + bias)**. Where activation is the activation function passed as the activation argument (if not None), kernel is a weights matrix created by the layer, and bias is a bias vector created by the layer (only if use_bias is True).

In [0]:
a = tf.placeholder(tf.float32, shape=[None, 3])

m = tf.layers.Dense(units=2)
print(m)
b = m(a)

# another way
# = tf.layers.Dense(units=1)(a)

# functional way
# b = tf.layers.dense(a, units=1)

print(b)

Tensor("dense/BiasAdd:0", shape=(?, 2), dtype=float32)


As specified before, the `tf.layers.Layer` objects may use `tf.Variable` objects, its usage requires a step before this, these variables have to be **initialized** with the `tf.global_variables_initializer` function, like other tensorflow actions, it returns a handler to be executed in a session where the variables will be used.

*The *`tf.global_variables_initializer`* shall be executed after all *`tf.Variable`* declaraions, if a *`tf.Variable`* is created after the session runs the initializer, an exception will be thrown because of the unitialized variable.*

#### Executing Layers

After the initialization of the layer's tensor, it works and can be evaluated like any other tensor.

In [0]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer()) # initializing layer variables
    print(session.run(b, feed_dict={a: [[0, 0, 0], [10, 10, 10], [20, 20, 20], [30, 30, 30], [50, 50, 50]]}))

[[  0.          0.       ]
 [  4.1490574 -14.245133 ]
 [  8.298115  -28.490267 ]
 [ 12.44717   -42.735397 ]
 [ 20.745285  -71.22567  ]]


### Training

#### Defining the Data

As the most common training APIs, tensorflow needs two vector inputs with the data and its classes. The `data` tensor provides the elements to train the network, its elements shall have the same size. The `true` tensor provides the elements classes, that also need to have the same size.

In [0]:
data = tf.constant([[3.2, 2.4, 3.0, 2.6, 2.9], [2.5, 2.1, 2.2, 2.7, 2.2], [1.0, 2.0, 1.8, 2.2, 0.4]], dtype=tf.float32)
true = tf.constant([[0, 0],[0, 1], [1, 0]], dtype=tf.float32)

print(data, true)

Tensor("Const_2:0", shape=(3, 5), dtype=float32) Tensor("Const_3:0", shape=(3, 2), dtype=float32)


#### Defining the Training Model

The training model have to be compatible with the data to be trainned, in this case, the layer created to compute the predicted elements classes (`pred`) shall provide a bidimenssional output (two neurons -> `units=2`). If the layer output doesn't match the `true` tensor elements sizes, the `pred` layer will work anyways but it can't be trainned because of the size disparity.

In [0]:
pred = tf.layers.dense(data, units=2) # functional shorcut

with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    print(session.run(pred))

[[2.767838  1.4355589]
 [2.4556906 1.3064704]
 [1.1662503 2.4545648]]


#### Loss Function

The loss function is a way to check the network classification performance. One of the most common loss function for regression problems is the MSE (Mean Squared Error), its implementation could be archived manually with some low level math operations but tensforflow provides a set of common loss functions in the `tf.losses` module.

In [0]:
loss = tf.losses.mean_squared_error(labels=true, predictions=pred)

with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    print(session.run(loss))

15.827122


#### Optimizing

With a defined training model and a loss function, we can now optimize whe network performance through optimizers. Optimizers are algorithms to minimize or maximize a loss function of a determined model with the provided data. TensorFlow provides optimizers that inherit the `tf.train.Optimizer` class. Those optimizers incrementally change models variables in order to minimize the loss.

The simplest optimization algorithm is Gradient Descent, it's implemented in the `tf.train.GradientDescentOptimizer`. It works by modifying each model variables according the magnitude of the derivative of loss with respect to that variable.

In [0]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    for i in range(10):
        _, loss_value = session.run((train, loss))
        print(loss_value)

9.573689
5.2769394
2.9919262
1.7756245
1.1270847
0.7801912
0.59357715
0.4921455
0.43600214
0.40395436


## TensorBoard

TensorFlow provides a utility called TensorBoard. One of TensorBoard's many capabilities is visualizing a computation graph. You can easily do this with a few simple commands.

First you save the computation graph to a TensorBoard summary file as follows:

In [0]:
a = tf.placeholder(tf.float32, shape=[None, 3])
m = tf.layers.Dense(units=2)

writer = tf.summary.FileWriter('./logs')
writer.add_graph(tf.get_default_graph())

!ls ./logs

events.out.tfevents.1540829411.441d69b13941


To start TensorBoard we have to run system commands in background, the jupyter `!` magic runs system commands but it is blocking, as the tensorboard is a webserver, it needs to run in background to allow the notebook continue running. It's possible to archieve this through the `get_ipython().system_raw("command")`, the command shall use system's shell notation to indicate its in background, in case of linux systems we use the operator `&`.

In [0]:
get_ipython().system_raw('tensorboard --logdir ./logs &')

Google Colaboratory does not provide any public URL to acess the TensorBoard through a tunnel, an easy way to do this is using [ngrok](https://ngrok.com/).

In [0]:
print('installing ngrok')
!curl --output 'ngrok.zip' -- 'https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip'
!unzip -o -- 'ngrok.zip'

print('starting ngrok')
get_ipython().system_raw('./ngrok http 6006 &')

print('getting ngrok tunnel address')
ngrok_tunnels = !curl 'http://localhost:4040/api/tunnels'
ngrok_tunnels = json.loads('\n'.join(ngrok_tunnels))

print(f'Access TensorBoard through ngrok at {ngrok_tunnels["tunnels"][0]["public_url"]}')

installing ngrok
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5237k  100 5237k    0     0  10.6M      0 --:--:-- --:--:-- --:--:-- 10.6M
Archive:  ngrok.zip
  inflating: ngrok                   
starting ngrok
getting ngrok tunnel address
Access TensorBoard through ngrok at http://34429041.ngrok.io


## Tensors

Tensors are the main elements manipulated and passed around to define computational graphs in tensorflow, all tensors inherit from the `tf.Tensor` class. A `tf.Tensor` is a representation of a single value or an array of any size, it can be a vector, matrix or, potentially, higher dimenssions. Internally, tensors are represented by n-dimenssional arrays with a fixed base data type for its elements **(this is only a representation, a tensor does not yield any value until it's computed in a `tf.Session`)**.

A `tf.Tensor` has the following properties:
* a data type (`tf.float32`, `np.int32`, `str` for example)
* a shape (defines the tensor rank)

Each element of a tensor object shall have the same type, and this type is always known. The shape need to be at least partially known. Most tensorflow operations generates tensors with fully-known type and shape, but in some cases, is only possible to find out the shape of a tensor at graph runtime.

TensorFlow has some special tensor types, they are:
* `tf.Variable` (the only one that allows value mutability, all other tensors are immutable, but they can provide different values in different `tf.Session.run` calls, e.g. generate a randpm value)
* `tf.constant` (it's a constant as the name says, with a predefined value)
* `tf.placeholder` (it can partially defined with its data and shape, but it have to be replaced by some input or tensor in an `tf.Session.run` call)
* `tf.SpaceTensor` (#TODO)

### Rank

A `tf.Tensor` rank is simply its number of dimenssions (the tensor rank is not the same as matrix rank).

| Rank | Mathematical entity |
| - | - |
| 0 | Scalar (strings are scalars) |
| 1 | Vector |
| 2 | Matrix |
| 3 | Vector of Matrices of Matrix of vectors |
| n | you get the idea |

#### Rank 0

In [0]:
# string = tf.constant('string', tf.string)
string = tf.constant('string') # the type is inferred
integer = tf.constant(123, tf.int16) # by default the type is tf.int32
floating = tf.Variable(1.23, tf.float64) # by default the type is tf.float32
complexx = tf.Variable(1.23 - 4.56j, tf.complex64)

print(string, integer, floating, complexx)

Tensor("Const_4:0", shape=(), dtype=string) Tensor("Const_5:0", shape=(), dtype=int16) <tf.Variable 'Variable:0' shape=() dtype=float32_ref> <tf.Variable 'Variable_1:0' shape=() dtype=complex128_ref>


#### Rank 1

In [0]:
string_vector = tf.constant(['string'], tf.string)
integer_vector = tf.Variable([1, 2, 3, 4, 5], tf.int32)
floating_vector  = tf.constant([3.14, 2.71], tf.float32)
complexx_vector = tf.Variable([1.23 - 4.56j, 7.8 - 9.0j], tf.complex64)

print(string_vector, integer_vector, floating_vector, complexx_vector)

Tensor("Const_6:0", shape=(1,), dtype=string) <tf.Variable 'Variable_2:0' shape=(5,) dtype=int32_ref> Tensor("Const_7:0", shape=(2,), dtype=float32) <tf.Variable 'Variable_3:0' shape=(2,) dtype=complex128_ref>


#### Higher Ranks

In [0]:
tensor0 = tf.constant([[1, 2], [3, 4]], tf.int16)
tensor1 = tf.constant([[1], [2], [3], [4]], tf.int16)
tensor2 = tf.constant([[[1, 2], [3, 4]], [[5, 6], [7, 8]]], tf.int16)
tensor3 = tf.constant([[[1], [2], [3], [4]], [[5], [6], [7], [8]]], tf.int16)
tensor4 = tf.constant([[[[1], [2]], [[3], [4]]], [[[5], [6]], [[7], [8]]]], tf.int16)
tensor5 = tf.zeros([10, 10, 256, 256, 4]) # using tensorflow api to generate

print(tensor0)
print(tensor1)
print(tensor2)
print(tensor3)
print(tensor4)
print(tensor5)

Tensor("Const_8:0", shape=(2, 2), dtype=int16)
Tensor("Const_9:0", shape=(4, 1), dtype=int16)
Tensor("Const_10:0", shape=(2, 2, 2), dtype=int16)
Tensor("Const_11:0", shape=(2, 4, 1), dtype=int16)
Tensor("Const_12:0", shape=(2, 2, 2, 1), dtype=int16)
Tensor("zeros:0", shape=(10, 10, 256, 256, 4), dtype=float32)


The rank of a tensor can be obtained through the `tf.rank` function, it will be available only through a `tf.Session.run` call.

In [0]:
with tf.Session() as session:
    print(session.run(tf.rank(tensor0)), end=' ')
    print(session.run(tf.rank(tensor1)), end=' ')
    print(session.run(tf.rank(tensor2)), end=' ')
    print(session.run(tf.rank(tensor3)), end=' ')
    print(session.run(tf.rank(tensor4)), end=' ')
    print(session.run(tf.rank(tensor5)), end=' ')

2 2 3 3 4 5 

### Slices

Tensors can be sliced in the computational graph like `numpy.ndarray` objects.

Rank 0 tensors can't be sliced because they already are a single element.

`Rank N (D0, D1, ... , Dn)` tensors behave the following way:
1. To access a single element, it's nessessary to provide `n (d0 < D0, d1 < D1, ... , dn < DN)` indices
2. Providing less indices like `n-k (d0 < D0, d1 < D1, ... , dn-k-1 < Dn-k-1, dn-k < Dn-k)` generates a element with D(Dn-k+1, Dn-k+2, ... , Dn) shape
3. Providing ranges for any `k (: or dkfrom:dkto where dkfrom <= dkto <= DK)` for subselecting a range of dimenssion elements instead of a single element
4. Combining the previous ones

In [0]:
rank5_tensor = tf.zeros([2, 4, 8, 16, 32])
print('rank 5 tensor for slicing:')
print(rank5_tensor)
print()

print('slicing single index in all dimensions:')
print(rank5_tensor[0, 2, 4, 6, 8])
print()

print('slicing single index with increasing missing dimensions:')
print(rank5_tensor[0, 2, 4, 6])
print(rank5_tensor[0, 2, 4])
print(rank5_tensor[0, 2])
print(rank5_tensor[0])
print()

print('slicing without selecting in all dimenssions:')
print(rank5_tensor[:, :, :, :, :])

print('slicing selecting the 2 first elements in all dimenssions:')
print(rank5_tensor[:2, :2, :2, :2, :2])
print(rank5_tensor[:2, :2, :, :2, :2])
print(rank5_tensor[:2, :2, 2, :2, :2])
print(rank5_tensor[:2, :, 2, :2])

rank 5 tensor for slicing:
Tensor("zeros_1:0", shape=(2, 4, 8, 16, 32), dtype=float32)

slicing single index in all dimensions:
Tensor("strided_slice:0", shape=(), dtype=float32)

slicing single index with increasing missing dimensions:
Tensor("strided_slice_1:0", shape=(32,), dtype=float32)
Tensor("strided_slice_2:0", shape=(16, 32), dtype=float32)
Tensor("strided_slice_3:0", shape=(8, 16, 32), dtype=float32)
Tensor("strided_slice_4:0", shape=(4, 8, 16, 32), dtype=float32)

slicing without selecting in all dimenssions:
Tensor("strided_slice_5:0", shape=(2, 4, 8, 16, 32), dtype=float32)
slicing selecting the 2 first elements in all dimenssions:
Tensor("strided_slice_6:0", shape=(2, 2, 2, 2, 2), dtype=float32)
Tensor("strided_slice_7:0", shape=(2, 2, 8, 2, 2), dtype=float32)
Tensor("strided_slice_8:0", shape=(2, 2, 2, 2), dtype=float32)
Tensor("strided_slice_9:0", shape=(2, 4, 2, 32), dtype=float32)


### Shape

As said  before, the shape is the size of each dimenssion of a tensor, this data is partially inferred by tensorflow during graph construction. These shapes can be represented by tensorflow via python list, tuple of ints or the `tf.TensorShape` object.

A `tf.Tensor` shape can be accessed through the `shape` property, that may return a partially defined shape during the graph construction, because they aren't fully known. Another way to get the shape, in this case the fully known, is by calling the `tf.shape` method in na `tf.Session`.

#### Changing a `tf.Tensor` Shape

A `tf.Tensor` shape can be changed using the `tf.reshape` function, there is contraint to reshape a tensor, the current tensor output shape shall be compatible with the new shape, it means the product of all shape dimension sizes of the current and new shapes have to be equal.

Sometimes the shape isn't fully known, for this tensorflow allows the use of -1 as one of the dimenssions, it will compute the dimenssion size based on the others dimenssions, but a integer number that override -1 shall exist or it will lead to errors.

In [0]:
rank3_tensor = tf.zeros([3, 4, 5]) # 3 * 4 * 5 = 60
matrix_a = tf.reshape(rank3_tensor, [6, 10]) # 6 * 10 = 60
matrix_b = tf.reshape(rank3_tensor, [4, -1]) # -1 tells reshape to calculate the size => 15

rank4_tensor_a = tf.reshape(rank3_tensor, [2, 2, 3, -1])
rank4_tensor_b = tf.reshape(rank3_tensor, [2, -1, 3, 5])
rank4_tensor_c = tf.reshape(rank3_tensor, [2, -1, -1, 5]) # two unkown dimenssions can not be computed

print(rank3_tensor)
print(matrix_a)
print(matrix_b)
print(rank4_tensor_a)
print(rank4_tensor_b)
print(rank4_tensor_c)

Tensor("zeros_2:0", shape=(3, 4, 5), dtype=float32)
Tensor("Reshape:0", shape=(6, 10), dtype=float32)
Tensor("Reshape_1:0", shape=(4, 15), dtype=float32)
Tensor("Reshape_2:0", shape=(2, 2, 3, 5), dtype=float32)
Tensor("Reshape_3:0", shape=(2, 2, 3, 5), dtype=float32)
Tensor("Reshape_4:0", shape=(2, ?, ?, 5), dtype=float32)


### Data Types

As explained before, tensors have shape and type, the later, even if the tensor represents any array, the types of the elements of this array shall be the same.
The following `tf.DTypes` are supported by `tf.Tensors`:
* `tf.float16`: 16-bit floating-point
* `tf.float32`: 32-bit floating-point
* `tf.float64`: 64-bit floating-point
* `tf.bfloat16`: 16-bit truncated floating-point
* `tf.complex64`: 64-bit complex
* `tf.complex128`: 128-bit complex
* `tf.int8`: 8-bit signed integer
* `tf.uint8`: 8-bit unsigned integer
* `tf.uint16`: 16-bit unsigned integer
* `tf.uint32`: 32-bit unsigned integer
* `tf.uint64`: 64-bit unsigned integer
* `tf.int16`: 16-bit signed integer
* `tf.int32`: 32-bit signed integer
* `tf.int64`: 64-bit signed integer
* `tf.bool`: Boolean
* `tf.string`: String
* `tf.qint8`: Quantized 8-bit signed integer
* `tf.quint8`: Quantized 8-bit unsigned integer
* `tf.qint16`: Quantized 16-bit signed integer
* `tf.quint16`: Quantized 16-bit unsigned integer
* `tf.qint32`: Quantized 32-bit signed integer
* `tf.resource`: Handle to a mutable resource
* `tf.variant`: Values of arbitrary types

A `tf.Tensor` type can be accessed through the `tf.Tensor.dtype`

In [0]:
int_tensor = tf.constant([1, 2, 3, 4]) # type infered
float_tensor = tf.constant([1, 2, 3, 4], dtype=tf.float64)
print(int_tensor.dtype, float_tensor.dtype)

<dtype: 'int32'> <dtype: 'float64'>


#### Casting `tf.Tensor`s DTypes

Tensors types can be casted using the `tf.cast` tensor, it takes a tensor of some dtype and outputs a new tensor with the same shape and the provided type.

In [0]:
float_tensor = tf.cast(tf.constant([1, 2, 3]), dtype=tf.float32)
print(float_tensor)

Tensor("Cast:0", shape=(3,), dtype=float32)


### Evaluating Tensors

After the computational graph construction, you can run the computation that yelds the values of any tensor in the graph.

The simplet way to evaluate a `tf.Tensor` is through the `tf.Tensor.eval` method, but this method has a requirement. As explained before, `tf.Tensor`s results are computed by `tf.Session`s, the same is necessary for the `tf.Tensor.eval` method, the **default** tesnsorflow session.

The default sesion can be started in two ways:
* Calling the `tf.InteractiveSession` function, or
* Setting an existing session as default using the `tf.Session.as_default` method (a with method is necessary)

Of course they can be evaluated by the `tf.Session.run` method

In [0]:
session = tf.InteractiveSession()
print(f'Is default session? {session == tf.get_default_session()}')

constant = tf.constant([1, 2, 3])
tensor = constant * constant
print(tensor.eval())

session.close()

Is default session? True
[1 4 9]


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
session = tf.Session()

with session.as_default() as d_session:
    print(f'Is d_session a default session? {d_session == tf.get_default_session()}')
    print(f'Is session a default session? {session == tf.get_default_session()}')
    print(d_session == session)

    constant = tf.constant([1, 2, 3])
    tensor = constant * constant
    print(tensor.eval()) # default session
    print(session.run(tensor)) # session run call

session.close()

Is d_session a default session? True
Is session a default session? True
True
[1 4 9]
[1 4 9]


## Variables